In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re
import undetected_chromedriver as uc
import time

# Login Bibliopass

Bibliopass is a remote publisher library owned by the University of Turin. Each remote publisher library may have a different login interface/scheme so you need to modify this login code. If you want to use a ScienceDirect subscription account, you also need to modify this login code. If you do not have access to log in to ScienceDirect, you cannot use this code (when we are not logged in, we may able to use this code but ScienceDirect usually only gives us a few pages of the search result so that the search result is not complete).

In [2]:
sciencedirect_url = "https://www-sciencedirect-com.bibliopass.unito.it/"

In [3]:
driver = uc.Chrome()

In [4]:
driver.get(sciencedirect_url)
time.sleep(5)

In [ ]:
try:
    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")
    username.send_keys("your_username")
    password.send_keys("your_password")
    driver.find_element_by_name("_eventId_proceed").click()
except:
    pass

# Call The Query Links Dataset

In [6]:
sciencedirect_links = pd.read_excel("./query-links/ScienceDirect_QueryBibliopass_Links.xlsx")

# Get Links

In [ ]:
# Get the paper links
list_source = []
list_querystring = []
list_querylinks = []
list_paperlinks = []
for index in range(0,len(sciencedirect_links["Query Links from All Attribute"])):
    # Get links from first pages
    driver.get(sciencedirect_links["Query Links from All Attribute"][index])
    time.sleep(5)
    hrefs = driver.find_elements_by_xpath("//*[@class='result-item-content']//*[@href]")
    for a in hrefs:
        link = a.get_attribute('href')
        if not "journal" in link:
            if not "pdf" in link:
                list_source.append("ScienceDirect")
                list_querystring.append(sciencedirect_links["Query"][index])
                list_querylinks.append(sciencedirect_links["Query Links from All Attribute"][index])
                list_paperlinks.append(link)

    # Get links from next pages
    while True:
      try:
        driver.find_element_by_link_text('next').send_keys('\n')
        time.sleep(5)
        hrefs = driver.find_elements_by_xpath("//*[@class='result-item-content']//*[@href]")
        for a in hrefs:
            link = a.get_attribute('href')
            if not "journal" in link:
                if not "pdf" in link:
                    list_source.append("ScienceDirect")
                    list_querystring.append(sciencedirect_links["Query"][index])
                    list_querylinks.append(sciencedirect_links["Query Links from All Attribute"][index])
                    list_paperlinks.append(link)
      except:
        break
driver.quit()

In [ ]:
# Save to xlsx
sciencedirect_paperlinks = pd.DataFrame({'Source': list_source,'Query': list_querystring, 'Query Links': list_querylinks, 'Paper Links': list_paperlinks})
sciencedirect_paperlinks.to_excel("./paper-links/PaperLinks_ScienceDirect_AllDuplicated.xlsx", index = False)
len(sciencedirect_paperlinks)

In [ ]:
# Drop Duplicate
sciencedirect_paperlinks_dd = sciencedirect_paperlinks.drop_duplicates(['Paper Links'], keep='first')
sciencedirect_paperlinks_dd.to_excel("./paper-links/PaperLinks_ScienceDirect.xlsx", index = False)
len(sciencedirect_paperlinks_dd)